In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
customer = pd.read_csv('customer.csv', sep=';')
transaction = pd.read_csv('transaction.csv', sep=';')

In [3]:
transaction = transaction.replace({np.nan: None})
customer = customer.replace({np.nan: None})

In [4]:
transaction['list_price']=transaction['list_price'].str.replace(',','.')
transaction['standard_cost']=transaction['standard_cost'].str.replace(',','.')

In [5]:
import psycopg2 

import psycopg2.extras as extras 
import pandas as pd 


def execute_values(conn, df, table): 
    tuples = [tuple(x) for x in df.to_numpy()] 
    cols = ','.join(list(df.columns)) 
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols) 
    cursor = conn.cursor() 
    try: 
        extras.execute_values(cursor, query, tuples) 
        conn.commit() 
        return True
    except (Exception, psycopg2.DatabaseError) as error: 
        print("Ошибка: %s" % error) 
        conn.rollback() 
        return False
    finally:
        cursor.close() 


conn = psycopg2.connect( 
	database="postgres", user='postgres', password='arsen', host='127.0.0.1', port='5433'
) 

In [6]:
execute_values(conn, customer, 'module2.customer') 

True

In [7]:
execute_values(conn, transaction, 'module2.transaction')

True

# Создание базы данных в PostgreSQL при помощи DBeaver

CREATE SCHEMA module2;<br>
<br>
<br>
CREATE table module2.transaction(<br>
transaction_id int4 PRIMARY KEY,<br>
product_id int4 NOT NULL,<br>
brand varchar(30) NULL,<br>
product_line varchar(30) NULL,<br>
product_class varchar(30) NULL,<br>
product_size varchar(30) null,<br>
customer_id int4 NULL,<br>
transaction_date timestamp NOT NULL,<br>
online_order varchar(30) NULL,<br>
order_status varchar(30) NOT NULL,<br>
list_price float4 NOT NULL,<br>
standard_cost float4 NULL,<br>
FOREIGN KEY(customer_id) REFERENCES module1.customer(customer_id)<br>
);<br>
<br>
CREATE table module2.customer(<br>
customer_id int4 PRIMARY KEY,<br>
first_name varchar(50) NULL,<br>
last_name varchar(50) NULL,<br>
gender varchar(30) NULL,<br>
DOB timestamp NULL,<br>
job_title varchar(50) NULL,<br>
job_industry_category varchar(50) NULL,<br>
wealth_segment varchar(50) NULL,<br>
deceased_indicator varchar(50) NULL,<br>
owns_car varchar(30) NULL,<br>
address varchar(50) NULL,<br>
postcode varchar(30) NULL,<br>
state varchar(30) NULL,<br>
country varchar(30) NULL,<br>
property_valuation int4 NULL<br>
);<br>

# Запросы к данным

### (1 балл) Вывести все уникальные бренды, у которых стандартная стоимость выше 1500 долларов.
select distinct brand from module2.transaction <br>where standard_cost>1500 and brand is not null;

### (1 балл) Вывести все подтвержденные транзакции за период '2017-04-01' по '2017-04-09' включительно.
select transaction_id  from module2.transaction <br>where order_status='Approved' and transaction_date between '2017-04-01' and '2017-04-09';


### (1 балл) Вывести все профессии у клиентов из сферы IT или Financial Services, которые начинаются с фразы 'Senior'.
select distinct job_title from module2.customer <br>where job_industry_category in ('IT', 'Financial Services') and job_title like 'Senior%';


### (1 балл) Вывести все бренды, которые закупают клиенты, работающие в сфере Financial Services
select distinct t.brand from module2.transaction t <br>
inner join  module2.customer c on t.customer_id = c.customer_id <br>
where c.job_industry_category='Financial Services' and t.brand is not null;


### (1 балл) Вывести 10 клиентов, которые оформили онлайн-заказ продукции из брендов 'Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles'.
select distinct t.customer_id, c.first_name, c.last_name  from module2.transaction t<br>
left join module2.customer c on t.customer_id = c.customer_id <br>
where t.brand in ('Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles') and t.online_order='true' limit 10 ;


### (1 балл) Вывести всех клиентов, у которых нет транзакций.
select customer_id, first_name, last_name from module2.customer <br>where customer_id not in (select customer_id from module2.transaction);


### (2 балла) Вывести всех клиентов из IT, у которых транзакции с максимальной стандартной стоимостью.
select distinct t.customer_id, c.first_name, c.last_name  from module2.transaction t <br>
inner join  module2.customer c on t.customer_id = c.customer_id <br>
where c.job_industry_category='IT' and t.standard_cost =(select max(standard_cost) from module2.transaction);


### (2 балла) Вывести всех клиентов из сферы IT и Health, у которых есть подтвержденные транзакции за период '2017-07-07' по '2017-07-17'.
select distinct t.customer_id, c.first_name, c.last_name from module2.transaction t <br>
inner join  module2.customer c on t.customer_id = c.customer_id <br>
where c.job_industry_category in ('IT','Health') and t.order_status='Approved' and t.transaction_date between '2017-07-07' and '2017-07-17';